In [1]:
%matplotlib inline

In [31]:
import os, sys

from utils import *
from vgg16 import Vgg16

from shutil import copyfile

In [6]:
%pwd

u'/home/ubuntu/courses/deeplearning1/wk'

In [10]:
current_dir = os.getcwd()
wk_dir = current_dir
data_dir = current_dir + "/../data/kg/cd-redux"

In [11]:
%cd $data_dir

/home/ubuntu/courses/deeplearning1/data/kg/cd-redux


In [16]:
%%bash
tree -d

.
├── test
└── train

2 directories


In [17]:
%ls -l test | wc -l

12501


In [18]:
%ls -l train | wc -l

25001


In [19]:
%mkdir -p train/cats
%mkdir -p train/dogs
%mkdir -p valid/cats
%mkdir -p valid/dogs
%mkdir -p test/unknown
%mkdir -p results

%mkdir -p sample/train/cats
%mkdir -p sample/train/dogs
%mkdir -p sample/valid/cats
%mkdir -p sample/valid/dogs

In [20]:
%%bash
tree -d

.
├── results
├── sample
│   ├── train
│   │   ├── cats
│   │   └── dogs
│   └── valid
│       ├── cats
│       └── dogs
├── test
│   └── unknown
├── train
│   ├── cats
│   └── dogs
└── valid
    ├── cats
    └── dogs

16 directories


In [29]:
%cd train

/home/ubuntu/courses/deeplearning1/data/kg/cd-redux/train


In [30]:
g = glob("*.jpg")
shuf = np.random.permutation(g)
for i in range(2000):
    os.rename(shuf[i], data_dir + "/valid/" + shuf[i])

In [34]:
g = glob("*.jpg")
shuf = np.random.permutation(g)
for i in range(200):
    copyfile(shuf[i], data_dir + "/sample/train/" + shuf[i])

In [35]:
%cd $data_dir/valid

/home/ubuntu/courses/deeplearning1/data/kg/cd-redux/valid


In [36]:
g = glob("*.jpg")
shuf = np.random.permutation(g)
for i in range(50):
    copyfile(shuf[i], data_dir + "/sample/valid/" + shuf[i])

In [37]:
%pwd

u'/home/ubuntu/courses/deeplearning1/data/kg/cd-redux/valid'

In [38]:
%cd $data_dir/train
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $data_dir/valid
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $data_dir/sample/train
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $data_dir/sample/valid
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

/home/ubuntu/courses/deeplearning1/data/kg/cd-redux/train
/home/ubuntu/courses/deeplearning1/data/kg/cd-redux/valid
/home/ubuntu/courses/deeplearning1/data/kg/cd-redux/sample/train
/home/ubuntu/courses/deeplearning1/data/kg/cd-redux/sample/valid


In [47]:
%cd $data_dir/test
%mv *.jpg unknown/

/home/ubuntu/courses/deeplearning1/data/kg/cd-redux/test
